In [1]:
import tensorflow as tf
from transformers import AutoTokenizer, AutoModel, AutoModelForQuestionAnswering
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers.schedules import CosineDecay
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [2]:
train_data = pd.read_csv('data/advanced_general_conversation_test_A.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
train_data

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
4520,4520,기타 괴롭힘 대화,"그 사람 또 늦게 왔어요.\n진짜? 지난번에도 지각하더니.\n아니, 무슨 일하는 태..."
4521,4521,기타 괴롭힘 대화,"네 친구 또 돈 빌려 달라고 했어.\n진짜? 지난번에도 안 갚았잖아.\n아니, 무슨..."
4522,4522,기타 괴롭힘 대화,"그 동료 또 실수했어요.\n진짜? 이번엔 또 뭐야?\n아니, 기본적인 것도 못 하는..."
4523,4523,기타 괴롭힘 대화,"걔 또 허세 부리더라.\n진짜? 저번에도 그러지 않았어?\n아니, 대체 왜 저러는 ..."


In [4]:
train_data['class'].unique()

array(['협박 대화', '기타 괴롭힘 대화', '갈취 대화', '직장 내 괴롭힘 대화', '일반 대화'],
      dtype=object)

In [5]:
label = {'협박 대화' : 0, '갈취 대화' : 1, '직장 내 괴롭힘 대화' :2, '기타 괴롭힘 대화' : 3, '일반 대화' : 4}

In [6]:
train_data['int_class'] = train_data['class'].map(label)

In [7]:
train_data

,idx,class,conversation,int_class
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,0
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,0
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,3
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,1
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,1
...,...,...,...,...
4520,4520,기타 괴롭힘 대화,"그 사람 또 늦게 왔어요.\n진짜? 지난번에도 지각하더니.\n아니, 무슨 일하는 태...",3
4521,4521,기타 괴롭힘 대화,"네 친구 또 돈 빌려 달라고 했어.\n진짜? 지난번에도 안 갚았잖아.\n아니, 무슨...",3
4522,4522,기타 괴롭힘 대화,"그 동료 또 실수했어요.\n진짜? 이번엔 또 뭐야?\n아니, 기본적인 것도 못 하는...",3
4523,4523,기타 괴롭힘 대화,"걔 또 허세 부리더라.\n진짜? 저번에도 그러지 않았어?\n아니, 대체 왜 저러는 ...",3


In [8]:
from transformers import BertTokenizer, TFBertModel

In [9]:
#MODEL_NAME = "monologg/koelectra-small-v2-distilled-korquad-384"
#tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
#model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME, num_labels=5, from_pt=True)

In [10]:
MODEL_NAME = "beomi/kcbert-base"
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = TFBertModel.from_pretrained(MODEL_NAME, num_labels=5, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [11]:
# 테스트 데이터 토큰화 (문장별 토큰 개수 확인)
tokenized_texts = [tokenizer.tokenize(text) for text in train_data['conversation']]

# 각 문장의 토큰 개수 계산
token_lengths = [len(tokens) for tokens in tokenized_texts]

# 최대 토큰 개수 찾기
max_token_length = max(token_lengths)

print(f"문장에서 가장 긴 토큰 개수: {max_token_length}")

문장에서 가장 긴 토큰 개수: 386


In [12]:
# 단어 사전 개수 확인
tokenizer.vocab_size

30000

In [13]:
# 특수 토큰 확인
print(tokenizer.cls_token)
print(tokenizer.sep_token)

[CLS]
[SEP]


In [14]:
# 최대 토큰 인텍스, 문장, 토큰 수 확인
max_token_idx = token_lengths.index(max(token_lengths))  # 가장 긴 문장의 인덱스
max_token_text = train_data['conversation'].iloc[max_token_idx]  # 해당 문장
max_token_count = token_lengths[max_token_idx]

In [15]:
max_token_text

'이게 어떻게 된거야.? 엄마 수술은 어떻게 된거냐고\n 엄마가 이제 괜찮다고 해서 딱 봐도 많이 좋아지셨잖아. 굳이 수술까진 할 필요 없어보여서.\n 뭐? 그걸 왜 누나가 정해? 누나가 의사야? 병원가서 정밀검사 받아보고 해야지. 큰 병이면 어쩌려고 그래?\n 아 늙어서 몸도 성하지 않는 사람 수술해서 뭐 어쩔건데????\n 그럼 내가 준 수술비는 어쨌는데. 급하다며 급하다고 해서 내가 돈 다 빼서 줬잖아. 그거 어딨냐고 !!!!!!!!!\n 그건 다른 급한 곳에 썼어.\n 하. 급한 곳? 뭔데 어디다 썼는데.\n 엄마 모시고 살면 돈 나가는 곳이 한 두 군데니? 이것 저것 급한 곳들 돈 썼지. 니가 말하면 알아?\n 이건 뭐야? 못 보던건데. 누나 또 명품샀어?\n 아 내놔!!!!!!!! 니가 뭘 알아. 아니야 이거 원래 있던거야.\n 하. 니가 그러고도 인간이야? 그게 어떤돈인데!!!!!!!!!!!! 내가 밤낮으로 잠도 못자가면서 번 돈 !!!!!! 엄마 수술비하라고 다 빼서 준 걸 니가 가방을 쳐 사는데 써???????????\n 아 시끄러워. 나 바빠 가야해.\n 이성을 잃은 듯 너같은 년이 죽어야하는데. 너같은 년은 살 가치가 없다. 너가 죽어 너가 !!!!!!!!!!!\n 왜.왜이래??! 미쳤어?\n 이제 도저히 못참겠다. 몇 년 동안 이게 몇 번째야? 이 짓거리 못하도록 그냥 널 병신 만드는게 낫겠다. 팔 다리를 부러뜨려야 밖으로 안싸돌아 다니지? 발목을 아주 잘라버려야겠어. 주변에 흉기를 찾는다\n 미.미쳤어 왜이렇게 흥분해?!?!?\n 공구함에 있던 망치를 들며 너 내가 오늘 가만 안둬. 미친 건 너야. 팔다리를 없애버리겠어.\n 꺄아아아'

In [16]:
max_token_count

386

In [17]:
MAX_LEN = max_token_count + 2 # 문장 최대 토큰 길이
NUM_LABELS = 5  # 분류할 클래스 개수
BATCH_SIZE = 8
EPOCHS = 10

In [18]:
# 텍스트를 토큰화하는 함수
def tokenize_texts(texts, labels):
    encodings = tokenizer(
        list(texts),  # 리스트 형태로 변환
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN, 
        return_tensors="tf"
    )
    return encodings["input_ids"], encodings["attention_mask"], tf.convert_to_tensor(labels, dtype=tf.int32)

In [19]:
# 데이터 분할 (8:2 비율로 train/val 나누기)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_data['conversation'], train_data['int_class'], test_size=0.2, random_state=42
)

In [20]:
# 데이터 토큰화
train_input_ids, train_attention_mask, train_labels = tokenize_texts(train_texts, train_labels)
val_input_ids, val_attention_mask, val_labels = tokenize_texts(val_texts, val_labels)

In [21]:
# TensorFlow Dataset 생성
train_dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": train_input_ids, "attention_mask": train_attention_mask}, train_labels)
).shuffle(len(train_texts)).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": val_input_ids, "attention_mask": val_attention_mask}, val_labels)
).batch(BATCH_SIZE)

In [22]:
# 데이터 변환 (train_data['conversation'], train_data['int_class'] 사용)
input_ids, attention_mask, label_tensor = tokenize_texts(train_data['conversation'], train_data['int_class'])

# TensorFlow Dataset 생성
dataset = tf.data.Dataset.from_tensor_slices(
    ({"input_ids": input_ids, "attention_mask": attention_mask}, label_tensor)
)

# 배치 및 섞기
dataset = dataset.shuffle(len(train_data)).batch(BATCH_SIZE)

In [23]:
from tensorflow.keras import backend as K

In [24]:
# F1 평가지표
'''
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=1)  # 가장 높은 확률값을 가진 클래스 선택
        self.precision.update_state(y_true, y_pred)
        self.recall.update_state(y_true, y_pred)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + K.epsilon()))  # F1-score 계산

    def reset_state(self):
        self.precision.reset_state()
        self.recall.reset_state()
        '''

In [25]:
input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")

# BERT 모델에 입력
bert_output = model(input_ids, attention_mask=attention_mask)[1]
x = Dropout(0.1)(bert_output)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
output = Dense(NUM_LABELS, activation="softmax")(x)  # 다중 클래스 분류

# 모델 정의
model = Model(inputs=[input_ids, attention_mask], outputs=output)

# 컴파일
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy"]
)

# 모델 구조 확인
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 388)]        0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 388)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 108918528   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dropout_37 (Dropout)            (None, 768)          0           tf_bert_model[0][1]          

In [26]:
# EarlyStopping 콜백 설정 (patience=2)
early_stopping = EarlyStopping(
    monitor="val_loss",  # 검증 데이터 손실 기준
    patience=3,  # 3epoch 동안 개선되지 않으면 종료
    restore_best_weights=True  # 가장 성능이 좋았던 가중치 복원
)

In [27]:
# ModelCheckpoint 설정 (최적 모델 저장)
model_checkpoint = ModelCheckpoint(
    "best_kobert_testA.h5",  # 저장할 모델 파일명
    monitor="val_loss",  # 검증 손실 기준으로 저장
    save_best_only=True,  # 가장 성능 좋은 모델만 저장
    save_weights_only=True,  # 가중치만 저장 (전체 모델 저장하려면 False)
    verbose=1  # 저장될 때 로그 출력
)

In [28]:
print(early_stopping)  # 정상적으로 설정되었는지 확인
print(model_checkpoint)

In [29]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/10
453/453 [==============================] - 361s 766ms/step - loss: 0.5161 - accuracy: 0.8144 - val_loss: 0.3682 - val_accuracy: 0.8652

Epoch 00001: val_loss improved from inf to 0.36817, saving model to best_kobert_testA.h5
Epoch 2/10
453/453 [==============================] - 345s 761ms/step - loss: 0.1959 - accuracy: 0.9370 - val_loss: 0.3283 - val_accuracy: 0.9039

Epoch 00002: val_loss improved from 0.36817 to 0.32827, saving model to best_kobert_testA.h5
Epoch 3/10
453/453 [==============================] - 345s 761ms/step - loss: 0.0806 - accuracy: 0.9754 - val_loss: 0.3507 - val_accuracy: 0.9072

Epoch 00003: val_loss did not improve from 0.32827
Epoch 4/10
453/453 [==============================] - 344s 760ms/step - loss: 0.0502 - accuracy: 0.9859 - val_loss: 0.3984 - val_accuracy: 0.9127

Epoch 00004: val_loss did not improve from 0.32827
Epoch 5/10
453/453 [==============================] - 344s 760ms/step - loss: 0.0310 - accuracy: 0.9912 - val_loss: 0.4960 - val

In [30]:
from sklearn.metrics import f1_score

In [31]:
# 검증 데이터 예측
val_predictions = model.predict({"input_ids": val_input_ids, "attention_mask": val_attention_mask})
predicted_classes = np.argmax(val_predictions, axis=1)  # 확률값을 가장 높은 클래스 인덱스로 변환

# F1-score 계산
f1 = f1_score(val_labels, predicted_classes, average="weighted")

print(f"검증 데이터 F1-score: {f1:.4f}")

검증 데이터 F1-score: 0.9032


In [32]:
def tokenize_test_texts(texts):
    encodings = tokenizer(
        list(texts),
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="tf"
    )
    return encodings["input_ids"], encodings["attention_mask"]

In [33]:
test_input_ids, test_attention_mask = tokenize_test_texts(test_data['text'])

In [34]:
# 모델 예측 수행
predictions = model.predict({"input_ids": test_input_ids, "attention_mask": test_attention_mask})

# 확률값을 가장 높은 클래스 인덱스로 변환
predicted_classes = np.argmax(predictions, axis=1)

# 결과를 데이터프레임에 추가
test_data['target'] = predicted_classes

In [35]:
# 예측된 결과 출력
print(test_data[['text', 'target']])

                                                  text  target
0    아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...       1
1    우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때?  네? 제가요? ...       2
2    너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...       2
3    이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...       4
4    아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...       3
..                                                 ...     ...
495  미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...       2
496  교수님 제 논문에 제 이름이 없나요?  아 무슨 논문말이야?  지난 번 냈던 논문이...       2
497  야 너  네 저요? 그래 너 왜요 돈좀 줘봐  돈 없어요 돈이 왜 없어 지갑은 폼이...       1
498  야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...       2
499  엄마 저 그 돈 안해주시면 정말 큰일나요.  이유도 말하지 않고. 몇번째니 경민아....       0

[500 rows x 2 columns]


In [36]:
test_data

,idx,text,target
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,1
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,2
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,2
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,4
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,3
...,...,...,...
495,t_495,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,2
496,t_496,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,2
497,t_497,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,1
498,t_498,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,2


In [37]:
# CSV로 저장
test_data[['idx', 'target']].to_csv("submission_kobert_testA.csv", index=False)

In [38]:
test_data['target'].value_counts()

3    145
2    142
1    110
0     81
4     22
Name: target, dtype: int64

In [39]:
test_data[test_data['target'] == 4]

,idx,text,target
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,4
57,t_057,아니 진짜 걍 지구 망할 거 같아 위쪽은 비 난리고 밑에는 비는 안 와도 공기 난...,4
68,t_068,옥상에서 사람이 떨어졌어요. 사람이 떨어졌다고요? 위치가 어디세요? 경기도 광주시 ...,4
82,t_082,"천안 두정동 제이빌라 인데요, 지금 화재가 발생했어요. 빌라 어디에 불이 났나요? ...",4
108,t_108,야 너 살 좀 빠졌냐 아니 그대론데 아 그래 좀 빠진 것 같네 요즘 운동하기는 하는...,4
116,t_116,예쁜이메리크리스마쑤 아직인뎅 원래 이브에하는거거든 카드써조 카드 어제 써서보냈는데 ...,4
139,t_139,지금 사장딸 없어서 에어팟끼고 동영상 보고 있어 사장 심부름 간다고 갔는데 아직도 ...,4
141,t_141,갬성 국 굳 왜 그래 나한테 엥 뭐가아 뻥치지 말라니 이 반응은 뭐지 그냥 인터넷에...,4
152,t_152,"시신을 발견했습니다. 위치가 어디시죠? 인천 영흥도인근 해상인데요, 검은색 옷을 입...",4
155,t_155,여부 안녕 오늘도 너무 행복했어 조심히 들어가요 여부야다 여뷰 바보 집 잘들어가써 ...,4


In [40]:

# 긴 텍스트 출력 제한 해제
pd.options.display.max_colwidth = None  

# 데이터프레임 출력
print(train_data["conversation"])

0                                                                                                                                                                                                                                   지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해? 진짜 죽여버리고 싶게.\n 정말 잘못했습니다.\n 너가 선택해. 너가 죽을래 네 가족을 죽여줄까.\n 죄송합니다. 정말 잘못했습니다.\n 너에게는 선택권이 없어. 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야.\n 선택 못하겠습니다. 한번만 도와주세요.\n 그냥 다 죽여버려야겠군. 이의 없지?\n 제발 도와주세요.
1                                                                                                                                                                                                                                                                                         길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 한번만 더 얘기한다.\n장난전화 걸지 마시죠. \n9시 40분 마트에 폭발물이 터지면 다 죽는거야. \n장난전화는 업무방해죄에 해당됩니다.\n판단은 너에게 달려있다. 길동경찰서에도 폭발물 터지면 꽤나 재미있겠지.\n선생님 진정하세요.\n난 이야기했어. 경고했다는 말이야.
2                                       

In [43]:
# 긴 텍스트 출력 제한 해제
pd.options.display.max_colwidth = None  
# 예측된 결과 출력
display(test_data[test_data['target'] == 4])

,idx,text,target
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 요즘 듣는 것도 들어봐 음 난 좀 별론데 좋을 줄 알았는데 아쉽네 내 취향은 아닌 듯 배고프다 밥이나 먹으러 가자 그래,4
57,t_057,아니 진짜 걍 지구 망할 거 같아 위쪽은 비 난리고 밑에는 비는 안 와도 공기 난리고 이제 또 비오는 너무 많이와 진짜 망할려고이러나 그러게 말이여 진짜 몇년 전부터 미세먼지도 난리고 원래는 봄에만 황사처럼 공기 안 좋았잖아 근데 요즘엔 걍 사계절 내내 안 좋으니까 약간 더 무서운 건 또 거기에 다 적응한 거임 일단 나부터 적응 다함 공기 안 좋은 거 그러려니해 아 개인정 어쩌다 이렇게 됐는지 진짜 예전이 그립다 초딩 때라던가 그냥 벌받는거 같기두 하구 마잡 그래서 나 늦은 감 있지만 이제라도 일회용 줄이려고 뭔가 이젠 다들 좀 경각심 느껴야 할 거 같음\n,4
68,t_068,"옥상에서 사람이 떨어졌어요. 사람이 떨어졌다고요? 위치가 어디세요? 경기도 광주시 오포읍 문형리 행복 아파트인데요, 사람이 떨어졌어요. 몇층에서 떨어졌는지는 보셨나요? 아니요 어디서 떨어졌는지는 못봤어요. 알겠습니다. 잠시만 기다려 주세요.",4
82,t_082,"천안 두정동 제이빌라 인데요, 지금 화재가 발생했어요. 빌라 어디에 불이 났나요? 1층 주차장에서 불이 나고 있어요. 그러면 지금 다친 사람도 있나요? 아직까지는 없는거 같아요. 알겠습니다. 지금 119 소방대를 보내도록 하겠습니다.",4
108,t_108,야 너 살 좀 빠졌냐 아니 그대론데 아 그래 좀 빠진 것 같네 요즘 운동하기는 하는데 잘 안빠져 일주일에 몇번 가 나 두번정도 별로 안가네 더 많이 가야지 그런가 겨울이면 원래 더 안빠지는거 아냐 에이 그런가 모르겠네 더 열심히 해봐 알았어 고맙다,4
116,t_116,예쁜이메리크리스마쑤 아직인뎅 원래 이브에하는거거든 카드써조 카드 어제 써서보냈는데 어디 카톡선물하면서 써서 보냈어 안봤나보네 그거카드아닌데\n,4
139,t_139,지금 사장딸 없어서 에어팟끼고 동영상 보고 있어 사장 심부름 간다고 갔는데 아직도 걔 땡땡이치네 바로 집갔을수도 있고 하 상상하니까 열받네 아냐 오늘 금요일이잖아 우리 즐겁게 지내자 알겠엉\n,4
141,t_141,갬성 국 굳 왜 그래 나한테 엥 뭐가아 뻥치지 말라니 이 반응은 뭐지 그냥 인터넷에 도는 짤 같았어 입맛감퇴짤 짤 제조기인가 멋졍 머래 나 곧 노래하러 나간다 지금 백스테이지임 오롯\n,4
152,t_152,"시신을 발견했습니다. 위치가 어디시죠? 인천 영흥도인근 해상인데요, 검은색 옷을 입은 남자가 떠있는것을 발견했어요. 그럼 가까운 지상으로 시신을 가져와주실수 있나요? 네 영흥도 부둣가로 갈게요 알겠습니다.",4
155,t_155,여부 안녕 오늘도 너무 행복했어 조심히 들어가요 여부야다 여뷰 바보 집 잘들어가써 이모티콘 넹 여뷰 나두 오늘 너무 행복해써 사랑해 나듀 나 사랑해 좋겟당\n,4
